In [1]:
using Revise
using PastaQ
using ITensors
using Random
using Printf
using OptimKit
using Zygote
using Zygote: ChainRulesCore
using BenchmarkTools

import mVQE
using mVQE.ITensorsExtension: projective_measurement
using mVQE.StateFactory: random_MPS, infinite_temp_MPO
using mVQE.Layers: Rylayer, CXlayer, Rxlayer
using mVQE.Circuits: runcircuit, VariationalCircuitRy, VariationalMeasurement, initialize_circuit, generate_circuit
using mVQE: loss, optimize_and_evolve

In [2]:
function hamiltonian(N, h)
    os = OpSum()
  
    os += -J, "Z", 1, "Z", N
    os += -J, "X", 1, "X", N
    
    os += -h, "Z", 2
    
    return os
end

hamiltonian (generic function with 1 method)

In [3]:
ancillas = 1
ancillas_indices = [2]
N = 2 + ancillas   # number of qubits
J = 1.0  # Ising exchange interaction
h = 1.  # transverse magnetic field

# Hilbert space
hilbert = qubits(N)

# build MPO "cost function"
H = MPO(hamiltonian(N, 0.1), hilbert);
H2 = MPO(hamiltonian(N, 0.), hilbert);

In [4]:
# # find ground state with DMRG

# nsweeps = 10
# maxdims = [10, 20, 30, 50, 100]
# cutoff_ = 1e-10

# start_mps = randomMPS(hilbert, linkdims=10)
# Edmrg, Φ = dmrg(H, start_mps; outputlevel=0, nsweeps, maxdims, cutoff_);
# @printf("\nGround state energy from DMRG: %.10f\n\n", Edmrg)

In [5]:
# contract(Φ.data).tensor[:, 1, :]

## Variational ansatz

In [6]:
depth = 5

cutoffs = 1e-8
maxdims = 50

Random.seed!(1234)

# run VQE using BFGS optimization
optimizer = LBFGS(; maxiter=50, verbosity=1)

LBFGS{Float64, HagerZhangLineSearch{Rational{Int64}}}(8, 50, 1.0e-8, true, HagerZhangLineSearch{Rational{Int64}}(1//10, 9//10, 1//1000000, 1//2, 2//3, 5//1, 9223372036854775807, -1), 1)

# Pure States

In [7]:
ψs, states = random_MPS(hilbert, 2; ancilla_indices=2)
ψs = [randomMPS(hilbert, linkdims=10) for _ in 1:10]

loss_value, θs, ψs_n, niter = optimize_and_evolve(3, [2], ψs, H, variationalcircuit, depth; optimizer=optimizer, verbose=true);

LoadError: UndefVarError: variationalcircuit not defined

In [45]:
ψ_init = randomMPS(hilbert, linkdims=10)

MPS
[1] ((dim=2|id=581|"Qubit,Site,n=1"), (dim=4|id=54|"Link,l=1"))
[2] ((dim=4|id=54|"Link,l=1"), (dim=2|id=766|"Qubit,Site,n=2"), (dim=2|id=525|"Link,l=2"))
[3] ((dim=2|id=525|"Link,l=2"), (dim=2|id=422|"Qubit,Site,n=3"))


In [194]:
ψ = ψ_init
for i in 1:3
    circuit = variationalcircuit(N, depth, θs[i])
    #runcircuit(ψ, circuit; noise=noisemodel)
    ψ = runcircuit(ψ, circuit)
    println(loss(ψ, H2))
    ψ, m = projective_measurement(ψ, indices=[2], reset=1)
    
end

0.14654935978031258
-1.0937576757009784
-2.0000000000000004


In [179]:
noisemodel = (1 => ("depolarizing", (p = 0.,)), 
              2 => ("depolarizing", (p = 0.,)))
ρ = ψ_init
for i in 1:3
    circuit = variationalcircuit(N, depth, θs[i])
    ρ = runcircuit(ρ, circuit; noise=noisemodel)
    ρ, = projective_measurement(ρ, indices=[2], reset=1)
    println(loss(ρ, H2))
end

0.14654935978031308
-1.2098042695580364
-1.9999999999999947


## Pure state with ADAM Optimizer

In [14]:
using Flux
using Flux: update!

W = rand(2, 5)
b = rand(2)

predict(x) = (W * x) .+ b
loss_(x, y) = sum((predict(x) .- y).^2)

x, y = rand(5), rand(2) # Dummy data
l = loss_(x, y) # ~ 3

θ = Flux.params(W, b)
grads = gradient(() -> loss_(x, y), θ)

Grads(...)

In [15]:
loss, pullback = Zygote.pullback(ps) do
    preds = discriminator(all_data)
    loss = Flux.Losses.binarycrossentropy(preds, all_target)
end

In [17]:
grads[p]

LoadError: UndefVarError: p not defined

In [18]:
opt = Descent(0.1) # Gradient descent with learning rate 0.1

for p in (W, b)
  update!(opt, p, grads[p])
    println(p)
end

[0.78503577423136 0.04337504802829825 -0.1446791993932456 -0.22970440152262145 0.874378559337465; 0.6463273485542164 0.37433591118485243 0.1517483714635025 0.07005621813601606 0.08632429511679768]
[0.09743861100875478, 0.10611696109022106]


# Mixed States

In [5]:
model = VariationalCircuitRy(N, depth)
model

VariationalCircuitRy(N=3, depth=5)

In [6]:
ρ = infinite_temp_MPO(hilbert)
depth = 5
k = 3
loss_value, θs, ψs_n, niter = optimize_and_evolve(k, [2], ρ, H, model, depth; optimizer=optimizer, verbose=true);

iter: 1
Loss: -1.0000000000000004



┌ Info: LBFGS: converged after 16 iterations: f = -1.000000000000, ‖∇f‖ = 6.9690e-09
└ @ OptimKit /home/leinad/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:138


iter: 2
Loss: -2.0000000000000018



┌ Info: LBFGS: converged after 19 iterations: f = -2.000000000000, ‖∇f‖ = 9.5182e-09
└ @ OptimKit /home/leinad/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:138


iter: 3
Loss: -2.1000000000000028



┌ Info: LBFGS: converged after 22 iterations: f = -2.100000000000, ‖∇f‖ = 2.8220e-09
└ @ OptimKit /home/leinad/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:138


In [61]:
ψ_init = [MPO(randomMPS(hilbert, linkdims=10)) for i in 1:1000];
ψ_init, = projective_measurement(ψ_init, indices=[2], reset=1);

In [62]:
ψ = ψ_init
for i in 1:k
    circuit = variationalcircuit(N, depth, θs[i])
    #runcircuit(ψ, circuit; noise=noisemodel)
    ψ = runcircuit(ψ, circuit)
    println(loss(ψ, H2))
    ψ, = projective_measurement(ψ, indices=[2], reset=1)
end

-0.9939841063631942
-1.492355160705006
-1.9999999999995435


In [ ]:
ψ = ψ_init
for i in 1:k
    circuit = variationalcircuit(N, depth, θs[i])
    #runcircuit(ψ, circuit; noise=noisemodel)
    ψ = runcircuit(ψ, circuit)
    println(loss(ψ, H2))
    ψ, = projective_measurement(ψ, indices=[2], reset=1)
end

In [7]:
m_model = MeasurementVariationalCircuitRy(N, depth, k, [2])

MeasurementVariationalCircuitRy(3, 5, 3, [2])

In [8]:
circuit = generate_circuit(m_model, θs)
ρ = infinite_temp_MPO(hilbert)
ρ, = projective_measurement(ρ; indices=[2], reset=1)

ρ = runcircuit(ρ, circuit)
println(loss(ρ, H2))

-2.000000000000003


with errors

In [14]:
noise = (1 => ("depolarizing", (p = 0.01,)), 
         2 => ("depolarizing", (p = 0.05,)))

ρ = infinite_temp_MPO(hilbert)
ρ, = projective_measurement(ρ, indices=[2], reset=1)

model = VariationalCircuitRy(N, depth)

for i in 1:k
    circuit = generate_circuit(model, θs[mod1(i, k)])
    ρ = runcircuit(ρ, circuit; noise)
    println(loss(ρ, H2))
    
    ρ, = projective_measurement(ρ, indices=[2], reset=1)
end

-0.6559344583190393
-0.8680891669358506
-0.9620536312358301


In [16]:
circuit = generate_circuit(m_model, θs)
ρ = infinite_temp_MPO(hilbert)
ρ, = projective_measurement(ρ; indices=[2], reset=1)

ρ = runcircuit(ρ, circuit; noise)
println(loss(ρ, H2))

-0.9620536312358299


# With errors

In [21]:
optimizer = LBFGS(; maxiter=100, verbosity=1)

LBFGS{Float64, HagerZhangLineSearch{Rational{Int64}}}(8, 100, 1.0e-8, true, HagerZhangLineSearch{Rational{Int64}}(1//10, 9//10, 1//1000000, 1//2, 2//3, 5//1, 9223372036854775807, -1), 1)

In [280]:
noise = (1 => ("depolarizing", (p = 0.01,)), 
         2 => ("depolarizing", (p = 0.0,)))


ρ = infinite_temp_MPO(hilbert)
ρ, = projective_measurement(ρ; indices=[2], reset=1);
model = VariationalCircuitRy(N, depth)
oss_value, θs_error, ρ, niter = optimize_and_evolve(k, [2], ρ, H, model, depth; optimizer=optimizer, verbose=true, noise);

iter: 1
Loss: -0.8176891388457767



┌ Info: LBFGS: converged after 53 iterations: f = -0.817689138846, ‖∇f‖ = 7.5900e-09
└ @ OptimKit /home/leinad/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:138


iter: 2
Loss: -1.477389996423896



┌ Warning: LBFGS: not converged to requested tol: f = -1.477389996424, ‖∇f‖ = 6.9892e-07
└ @ OptimKit /home/leinad/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:141


iter: 3
Loss: -1.5345133093909427



┌ Info: LBFGS: converged after 96 iterations: f = -1.534513309391, ‖∇f‖ = 8.5252e-09
└ @ OptimKit /home/leinad/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:138


iter: 4
Loss: -1.5333550231146056



┌ Info: LBFGS: converged after 82 iterations: f = -1.533355023115, ‖∇f‖ = 3.7717e-09
└ @ OptimKit /home/leinad/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:138


In [27]:
noise = (1 => ("depolarizing", (p = 0.01,)), 
         2 => ("depolarizing", (p = 0.01,)))

ψ = infinite_temp_MPO(hilbert)
ψ, = projective_measurement(ψ, indices=[2], reset=1)

for i in 1:k
    circuit = generate_circuit(model, θs_error[mod1(i, k)])
    ψ = runcircuit(ψ, circuit; noise)
    println(loss(ψ, H2))
    
    ψ, = projective_measurement(ψ, indices=[2], reset=1)
end

-0.6559344583190392
-0.6559344583190422
-1.0804460443562742


In [25]:
circuit = generate_circuit(m_model, θs_error)
ρ = infinite_temp_MPO(hilbert)
ρ, = projective_measurement(ρ; indices=[2], reset=1)

ρ = runcircuit(ρ, circuit; noise)
println(loss(ρ, H2))

-1.0804460443562744


# Optimize through the entire loop

## Test gradeints

In [7]:
ρ = infinite_temp_MPO(hilbert)
ρ, = projective_measurement(ρ; indices=[2], reset=1);
noise = (1 => ("depolarizing", (p = 0.01,)), 
         2 => ("depolarizing", (p = 0.05,)));

In [8]:
model_m = MeasurementVariationalCircuitRy(N, depth, 1, [2])
model = VariationalCircuitRy(N, depth)

VariationalCircuitRy(N=3, depth=5)

In [7]:
θ = mVQE.initialize_circuit(model)

5-element Vector{Vector{Float64}}:
 [2.048172193278203, 3.449790032588226, 1.3734204579032128]
 [5.618709735484967, 2.218665895883152, 2.477179531823963]
 [5.988658654067139, 4.998568891921718, 3.105463501721357]
 [4.702430269195866, 3.633138470776425, 4.573750504288762]
 [0.04679720270180084, 1.2527202123030283, 2.7598459521276157]

In [8]:
loss(ρ, H2, model, θ; noise), loss(ρ, H2, model_m, [θ]; noise)

LoadError: MethodError: no method matching loss(::Nothing, ::MPO)
[0mClosest candidates are:
[0m  loss([91m::Union{Vector{MPO}, Vector{MPS}}[39m, ::MPO; kwargs...) at ~/workprojects/mVQE/mVQE/src/mVQE.jl:155
[0m  loss([91m::MPS[39m, ::MPO; kwargs...) at ~/workprojects/mVQE/mVQE/src/mVQE.jl:142
[0m  loss([91m::MPO[39m, ::MPO; kwargs...) at ~/workprojects/mVQE/mVQE/src/mVQE.jl:146
[0m  ...

In [9]:
loss_(θ) =  loss(ρ, H2, model, θ; noise)

function loss_and_grad(θ)
    y, (∇,) = withgradient(loss_, θ)
    return y, ∇
end

N = length(ρ)

grad1 = loss_and_grad(θ)

(0.07378024887946513, [[-0.04163530941548421, 0.1908551329712285, -8.829742492721948e-16], [-0.041635309415484476, 0.07291006857904545, 0.06596025862809346], [0.04682389084370242, 0.040674767093854924, 0.06596025862809415], [0.046823890843702476, -0.030264804143833174, 0.024571521869207313], [-0.024571521869207365, -6.938893903907228e-18, 0.02457152186920737]])

In [159]:
loss_(θ) =  loss(ρ, H2, model_m, [θ]; noise)

function loss_and_grad(θ)
    y, (∇,) = withgradient(loss_, θ)
    return y, ∇
end

N = length(ρ)

v, grad2 = loss_and_grad(θ)

(0.24529037630806977, [[0.027159865458527388, 0.09721514535114369, 0.012100779192252747], [0.027159865458527055, -0.05959521716597113, 0.19939521165272742], [-0.22309145248859374, -0.39579782851518686, 0.19939521165272756], [-0.22309145248859313, 0.16398905839855255, 0.2504279649959604], [-0.2504279649959583, 5.273559366969494e-16, 0.2504279649959584]])

In [ ]:
loss_(θ) =  loss(ρ, H2, model_m, [θ]; noise)

function loss_and_grad(θ)
    y, (∇,) = withgradient(loss_, θ)
    return y, ∇
end

N = length(ρ)

v, grad2 = loss_and_grad(θ)

### Numerical check

In [62]:
depth = 6
model_m2 = MeasurementVariationalCircuitRy(N, depth, 2, [2])
model_m4 = VariationalCircuitRy(N, 2*depth)
θs = mVQE.initialize_circuit(model_m2);

In [63]:
circuit1 = generate_circuit(model_m2, θs)
circuit2 = generate_circuit(model_m4, vcat(θs[1], θs[2]))
ρ2 = runcircuit(ρ, circuit2.circ)
loss(ρ2, H2)

0.3135999911302023

In [64]:
ρ2 = runcircuit(ρ, vcat(circuit1.circ[1][1], circuit1.circ[2][1]))
loss(ρ2, H2)

0.3135999911302023

In [65]:
loss(ρ, H2, model_m2, θs)

0.31359999113020365

In [98]:
loss_(θ) = loss(ρ, H2, model_m2, θ)

function loss_and_grad(θ)
    y, (∇,) = withgradient(loss_, θ)
    return y, ∇
end

N = length(ρ)

lo, grad2 = loss_and_grad(θs);
lo

0.31359999113020365

In [99]:
l = 1
grad2[l][1][1]

-0.1510898738345156

In [100]:
h = 1e-11
θn = deepcopy(θs)
θn[l][1][1] = θn[l][1][1] + h
(loss_(θn) - loss_(θs))/h

-0.058247850986958845

In [240]:
loss_(θ) =  loss(ρ, H2, model_m4, θ)

function loss_and_grad(θ)
    y, (∇,) = withgradient(loss_, θ)
    return y, ∇
end

N = length(ρ)

lo, grad1 = loss_and_grad(vcat(θs[1], θs[2]));
lo

0.16579565266031346

In [241]:
grad1[(l-1)*5 + 1][1]

-0.06371863118819665

## Rest

In [107]:
noise = (1 => ("depolarizing", (p = 0.01,)), 
         2 => ("depolarizing", (p = 0.0,)))

optimizer = LBFGS(; maxiter=50, verbosity=1)

ρ = infinite_temp_MPO(hilbert)
ρ, = projective_measurement(ρ; indices=[2], reset=1)

m_model = mVQE.Circuit.OldMeasurementVariationalCircuitRy(N, depth, 1, [2])
#m_model = VariationalCircuitRy(N, depth)
@time loss_value, θs_error2, ρ, niter = optimize_and_evolve(ρ, H2, m_model, depth; optimizer, verbose=true, noise);

 46.238195 seconds (134.11 M allocations: 7.479 GiB, 7.17% gc time, 92.07% compilation time)


┌ Info: LBFGS: converged after 35 iterations: f = -0.604502977556, ‖∇f‖ = 6.2348e-09
└ @ OptimKit /home/leinad/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:138


In [108]:
circuit = generate_circuit(m_model, θs_error2)
ρ = infinite_temp_MPO(hilbert)
ρ, = projective_measurement(ρ; indices=[2], reset=1)

ρ = runcircuit(ρ, circuit; noise)
println(loss(ρ, H2))

-0.6045029775564075


In [114]:
circ = insertnoise(circuit.circ, noise);

In [119]:
circ = insertnoise(circuit.circ, noise);
t = gate(hilbert, circ[4])
t.tensor

Dim 1: (dim=2|id=606|"Qubit,Site,n=1")'
Dim 2: (dim=2|id=606|"Qubit,Site,n=1")
Dim 3: (dim=4|id=988|"kraus")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×4
[:, :, 1] =
 0.99498743710662 + 0.0im               0.0 + 0.0im
              0.0 + 0.0im  0.99498743710662 + 0.0im

[:, :, 2] =
                 0.0 + 0.0im  0.05773502691896258 + 0.0im
 0.05773502691896258 + 0.0im                  0.0 + 0.0im

[:, :, 3] =
 0.0 + 0.0im                  -0.0 - 0.05773502691896258im
 0.0 + 0.05773502691896258im   0.0 + 0.0im

[:, :, 4] =
 0.05773502691896258 + 0.0im                   0.0 + 0.0im
                 0.0 + 0.0im  -0.05773502691896258 + 0.0im

In [18]:
runcircuit(ρ, bc)

1.0e-1510000divide_and_conquertrue


MPO
[1] ((dim=2|id=743|"Qubit,Site,n=1")', (dim=2|id=743|"Qubit,Site,n=1"), (dim=3|id=902|"Link,n=1"))
[2] ((dim=2|id=412|"Qubit,Site,n=2")', (dim=3|id=902|"Link,n=1"), (dim=2|id=412|"Qubit,Site,n=2"), (dim=4|id=986|"Link,n=1"))
[3] ((dim=2|id=695|"Qubit,Site,n=3")', (dim=4|id=986|"Link,n=1"), (dim=2|id=695|"Qubit,Site,n=3"))


In [22]:
@which apply(bc, ρ; apply_dag=false)

(::ITensors.var"#product##kw")(::Any, ::typeof(product), As::Vector{ITensor}, ψ::ITensors.AbstractMPS) in ITensors at /home/leinad/.julia/packages/ITensors/PisGD/src/mps/abstractmps.jl:2152

In [30]:
using AbstractTrees
AbstractTrees.children(t::Type) = subtypes(t)

In [35]:
ITensors.AbstractMPS

ITensors.AbstractMPS

In [39]:
@which product(bc[1], ρ; apply_dag=false)

(var"#s48"::ITensors.var"#product##kw")(::Any, ::typeof(product), o::ITensor, ψ::ITensors.AbstractMPS) in ITensors at /home/leinad/.julia/packages/ITensors/PisGD/src/mps/abstractmps.jl:2027

# Reset Gates

In [7]:
noise = (1 => ("depolarizing", (p = 0.01,)), 
         2 => ("depolarizing", (p = 0.,)))

optimizer = LBFGS(; maxiter=100, verbosity=1)


N = 3
hilbert = qubits(N)
H = MPO(hamiltonian(N, 0.), hilbert);

ρ = infinite_temp_MPO(hilbert)
ρ, = projective_measurement(ρ; indices=[2], reset=1)

ψ = productstate(hilbert, fill(0, N))
ρ = outer(ψ, prime(ψ))

depth = 5
model = VariationalCircuitRy(N, depth)
reset = 1
k = 3
model = VariationalMeasurement(model, k, [2], reset)
@time loss_value, θs_error2, ρ, niter = optimize_and_evolve(ρ, H, model, depth; optimizer, verbose=true, noise);

 31.482786 seconds (44.65 M allocations: 7.348 GiB, 7.42% gc time, 0.30% compilation time)


┌ Warning: LBFGS: not converged to requested tol: f = -1.575814856524, ‖∇f‖ = 2.2716e-05
└ @ OptimKit /home/leinad/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:141


Test speed old vs new

In [93]:
ρ = infinite_temp_MPO(hilbert)
ρ, = projective_measurement(ρ; indices=[2], reset=1);

In [130]:
model_old = mVQE.Circuits.OldMeasurementVariationalCircuitRy(N, depth, k, [2])
loss(ρ, H, model_old, θs_error2; noise, maxdims=50)

-1.5971096499596662

In [129]:
model = VariationalCircuitRy(N, depth)
model = VariationalMeasurement(model, k, [2], reset)

loss(ρ, H, model, θs_error2; noise, maxdims=50)

-1.575814854525538

In [14]:
circuit1 = generate_circuit(model, θs_error2);
circuit2 = generate_circuit(model_old, θs_error2);

In [140]:
ρ1 = runcircuit(ρ, circuit1; noise)
loss(ρ1, H)

-1.575814854525538

In [141]:
ρ2 = runcircuit(ρ, circuit2; noise)
loss(ρ2, H)

-1.5971096499596662

In [144]:
t = (contract(ρ2.data) - contract(ρ1.data)).tensor;
mean(abs.(t.storage))

0.0008603783235126266

In [136]:
@btime loss(ρ, H, model_old, θs_error2; noise)

  30.423 ms (95918 allocations: 22.42 MiB)


-1.5971096499596662

In [137]:
@btime loss(ρ, H, model, θs_error2; noise)

  30.274 ms (97949 allocations: 22.91 MiB)


-1.575814854525538

# VQE

In [309]:
noise = (1 => ("depolarizing", (p = 0.01,)), 
         2 => ("depolarizing", (p = 0.01,)))

N = 3
hilbert = qubits(N)
H = MPO(hamiltonian(N, 0.), hilbert);

ψ = productstate(hilbert, fill(0, N))
ρ = outer(ψ, prime(ψ))
model = VariationalCircuitRy(N, depth*3)
oss_value, θs, ρ, niter = optimize_and_evolve(ρ, H, model, depth; optimizer=optimizer, verbose=true, noise);

┌ Warning: LBFGS: not converged to requested tol: f = -1.337337535800, ‖∇f‖ = 1.4628e-03
└ @ OptimKit /home/leinad/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:141
